## [Introduction to the LlamaHub](https://huggingface.co/learn/agents-course/unit2/llama-index/llama-hub#introduction-to-the-llamahub)

In [22]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="qwen2.5-coder:0.5b")

response = llm.complete("What is the capital of France?")
print(response)

The capital of France is Paris.


## [Components in LlamaIndex](https://huggingface.co/learn/agents-course/unit2/llama-index/components)

### Creating a RAG pipeline using components

In [23]:
# Setting up the persona database

from datasets import load_dataset
from pathlib import Path

dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")

Path("data").mkdir(parents=True, exist_ok=True)

for i, persona in enumerate(dataset):
    with open(Path("data") / f"persona_{i}.txt", "w") as f:
        f.write(persona["persona"])

In [24]:
# Loading and embedding persona documents

from llama_index.core import SimpleDirectoryReader, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline


reader = SimpleDirectoryReader(
    input_dir='data'
)
documents = reader.load_data()

print(f"Loaded {len(documents)} documents")
    
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_overlap=0),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ]
)

nodes = await pipeline.arun(documents=documents[:10])
print(f"Generated {len(nodes)} nodes")

Loaded 5000 documents
Generated 10 nodes


In [29]:
# Storing and indexing documents

import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex


db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection(name="alfred")
vector_store = ChromaVectorStore(chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ],
    vector_store=vector_store,
)
nodes = await pipeline.arun(documents=documents[:10])

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model,
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


In [33]:
# Querying a VectorStoreIndex with prompts and LLMs

llm = Ollama(model="qwen2.5-coder:0.5b")

query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
    verbose=True,
)

response = query_engine.query(
    "Respond using a persona that describes author and travel experiences?"
)
print(response)

The persona described is a web developer who has a strong focus on HTML, CSS, and website building.


In [34]:
# Evaluation & Observability

from llama_index.core.evaluation import FaithfulnessEvaluator

evaluator = FaithfulnessEvaluator(llm=llm)

eval_result = evaluator.evaluate_response(response=response)
print(eval_result.passing)

False
